Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [21]:
batch_size = 128
lam = 0.005
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + lam*tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [22]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 27.953396
Minibatch accuracy: 17.2%
Validation accuracy: 13.1%
Minibatch loss at step 500: 1.483679
Minibatch accuracy: 85.9%
Validation accuracy: 80.0%
Minibatch loss at step 1000: 0.819605
Minibatch accuracy: 79.7%
Validation accuracy: 81.8%
Minibatch loss at step 1500: 0.526913
Minibatch accuracy: 82.8%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 0.606993
Minibatch accuracy: 88.3%
Validation accuracy: 81.6%
Minibatch loss at step 2500: 0.742651
Minibatch accuracy: 80.5%
Validation accuracy: 81.4%
Minibatch loss at step 3000: 0.741505
Minibatch accuracy: 82.8%
Validation accuracy: 81.5%
Test accuracy: 89.0%


In [17]:
batch_size = 128
n_hidden = 2048
lam = 0.001
kp = 0.5
decay_steps = 2000
decay_rate = 0.96
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  keep_prob = tf.placeholder("float")
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  input_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, n_hidden]))
  input_biases = tf.Variable(tf.zeros([n_hidden]))
  hidden_weights = tf.Variable(
    tf.truncated_normal([n_hidden, num_labels]))
  hidden_biases = tf.Variable(tf.zeros([num_labels]))
  # Training computation.
  relus = tf.nn.relu(tf.matmul(tf_train_dataset, input_weights) + input_biases)
  relus = tf.nn.dropout(relus, keep_prob)
  #logits_train = tf.matmul(relus_drop, hidden_weights) + hidden_biases  
  logits = tf.matmul(relus, hidden_weights) + hidden_biases  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))\
    + lam*(tf.nn.l2_loss(input_weights) + tf.nn.l2_loss(hidden_weights))
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, decay_steps, decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_relus = tf.nn.relu(tf.matmul(tf_valid_dataset, input_weights) + input_biases)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_relus, hidden_weights) + hidden_biases)
  test_relus = tf.nn.relu(tf.matmul(tf_test_dataset, input_weights) + input_biases)
  test_prediction = tf.nn.softmax(tf.matmul(test_relus, hidden_weights) + hidden_biases)

In [18]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    #offset = (step * batch_size) % 5
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: kp}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1285.994751
Minibatch accuracy: 7.8%
Validation accuracy: 25.8%
Minibatch loss at step 500: 431.049622
Minibatch accuracy: 80.5%
Validation accuracy: 80.1%
Minibatch loss at step 1000: 249.140045
Minibatch accuracy: 72.7%
Validation accuracy: 82.7%
Minibatch loss at step 1500: 148.066208
Minibatch accuracy: 85.2%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 88.708267
Minibatch accuracy: 82.0%
Validation accuracy: 84.1%
Minibatch loss at step 2500: 54.489361
Minibatch accuracy: 78.9%
Validation accuracy: 85.0%
Minibatch loss at step 3000: 33.993320
Minibatch accuracy: 78.1%
Validation accuracy: 85.5%
Minibatch loss at step 3500: 21.576078
Minibatch accuracy: 82.0%
Validation accuracy: 86.2%
Minibatch loss at step 4000: 13.681285
Minibatch accuracy: 84.4%
Validation accuracy: 86.4%
Minibatch loss at step 4500: 8.848766
Minibatch accuracy: 86.7%
Validation accuracy: 86.1%
Minibatch loss at step 5000: 5.821907
Minibatch accuracy: 85.9%
Valid

Problem 2
----
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [40]:
batch_size = 128
n_hidden = 1024
n_hidden_2 = 512
lam = 0.0004
kp = 0.5
decay_steps = 15000
decay_rate = 0.98
num_steps = 3001
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  keep_prob = tf.placeholder("float")
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  input_weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, n_hidden], stddev = 0.1))
  input_biases = tf.Variable(tf.zeros([n_hidden]))
  hidden_weights = tf.Variable(
    tf.truncated_normal([n_hidden, n_hidden_2],stddev = 0.1))
  hidden_biases = tf.Variable(tf.zeros([n_hidden_2]))
  # Training computation.
  hidden_weights_2 = tf.Variable(
    tf.truncated_normal([n_hidden_2, num_labels]))
  hidden_biases_2 = tf.Variable(tf.zeros([num_labels]))
  
  def model(data):
    relus = tf.nn.relu(tf.matmul(data, input_weights) + input_biases)
    relus_2 = tf.nn.relu(tf.matmul(relus, hidden_weights)+ hidden_biases)
    relus_2 = tf.nn.dropout(relus_2, keep_prob)
    return relus_2
  # second layer. softmax followed by a smaller set of relus
  relus = model(tf_train_dataset)
  logits = tf.matmul(relus, hidden_weights_2) + hidden_biases_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))\
+ lam*(tf.nn.l2_loss(input_weights) + tf.nn.l2_loss(hidden_weights_2))
        


  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.088, global_step, decay_steps, decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_logits = tf.matmul(model(tf_valid_dataset), hidden_weights_2) + hidden_biases_2
  valid_prediction = tf.nn.softmax(valid_logits)
  test_logits = tf.matmul(model(tf_test_dataset), hidden_weights_2) + hidden_biases_2
  test_prediction = tf.nn.softmax(test_logits)

num_steps = 70001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:kp}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:1.0}), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:1.0}), test_labels))

Initialized
Minibatch loss at step 0: 64.354660
Minibatch accuracy: 8.6%
Validation accuracy: 35.7%
Minibatch loss at step 500: 2.672391
Minibatch accuracy: 74.2%
Validation accuracy: 81.0%
Minibatch loss at step 1000: 2.539128
Minibatch accuracy: 80.5%
Validation accuracy: 82.6%
Minibatch loss at step 1500: 2.314141
Minibatch accuracy: 83.6%
Validation accuracy: 83.1%
Minibatch loss at step 2000: 2.126428
Minibatch accuracy: 89.8%
Validation accuracy: 84.3%
Minibatch loss at step 2500: 2.255933
Minibatch accuracy: 81.2%
Validation accuracy: 84.6%
Minibatch loss at step 3000: 2.187811
Minibatch accuracy: 85.2%
Validation accuracy: 85.2%
Minibatch loss at step 3500: 2.045430
Minibatch accuracy: 86.7%
Validation accuracy: 85.6%
Minibatch loss at step 4000: 2.012513
Minibatch accuracy: 87.5%
Validation accuracy: 86.0%
Minibatch loss at step 4500: 1.943218
Minibatch accuracy: 84.4%
Validation accuracy: 86.4%
Minibatch loss at step 5000: 1.915604
Minibatch accuracy: 85.9%
Validation accurac

In [38]:
print(hidden_weights)

### Final Accuracy
96.1%. 